In [8]:
import pandas as pd
import numpy as np

# Define file names
file_names = ['HR_01.txt', 'HR_02.txt', 'SL_01.txt', 'SL_02.txt', 'SK_01.txt', 'SK_02.txt']

# Function to load and clean data
def load_clean_data(file_name):
    data = pd.read_csv(file_name, delimiter=',', skiprows=4)  # Adjust skiprows if needed
    numeric_data = data.select_dtypes(include=[np.number])
    return numeric_data.to_numpy()

# Load data from all files
eeg_data = [load_clean_data(file) for file in file_names]

# Print shapes of loaded data
for i, data in enumerate(eeg_data):
    print(f"Shape of data from {file_names[i]}: {data.shape}")


Shape of data from HR_01.txt: (30355, 24)
Shape of data from HR_02.txt: (30842, 24)
Shape of data from SL_01.txt: (4839, 24)
Shape of data from SL_02.txt: (30180, 24)
Shape of data from SK_01.txt: (34488, 24)
Shape of data from SK_02.txt: (30640, 24)


In [9]:
from scipy.signal import butter, filtfilt

# Define a bandpass filter
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Apply the bandpass filter
def apply_bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    return filtfilt(b, a, data, axis=0)

# Filter the data for each file
sampling_rate = 256  # Example sampling rate, adjust if needed
lowcut = 0.5  # Hz
highcut = 50  # Hz

filtered_eeg_data = [apply_bandpass_filter(data, lowcut, highcut, sampling_rate) for data in eeg_data]


In [10]:
# Compute FFT
def compute_fft(data, fs):
    n = len(data)
    freqs = np.fft.fftfreq(n, 1/fs)
    fft_values = np.fft.fft(data)
    fft_magnitude = np.abs(fft_values)
    return freqs[:n // 2], fft_magnitude[:n // 2]  # Return only positive frequencies

# Define a function to calculate power in specific frequency bands
def bandpower(data, freqs, low, high):
    idx_band = np.logical_and(freqs >= low, freqs <= high)
    return np.mean(data[idx_band])

# Define the EEG bands
bands = {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 12), 'beta': (12, 30), 'gamma': (30, 50)}

# Extract power in each band for each file
features = []
labels = []

for file_name in file_names:
    # Load data
    data = load_clean_data(file_name)
    
    # Apply filtering
    filtered_data = apply_bandpass_filter(data, lowcut, highcut, sampling_rate)
    
    # Extract features
    feature_vector = []
    for channel in range(filtered_data.shape[1]):
        freqs, fft_magnitude = compute_fft(filtered_data[:, channel], sampling_rate)
        band_features = [bandpower(fft_magnitude, freqs, *bands[band_name]) for band_name in bands]
        feature_vector.extend(band_features)
    
    features.append(feature_vector)
    
    # Label: Assume HR files are class 0, SL files are class 1
    if 'HR' in file_name:
        labels.append(0)
    else:
        labels.append(1)

# Convert to NumPy arrays
features = np.array(features)
labels = np.array(labels)

# Print updated shapes
print("Updated Features shape:", features.shape)
print("Updated Labels shape:", labels.shape)


Updated Features shape: (6, 120)
Updated Labels shape: (6,)


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train a Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict the test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest accuracy: {accuracy_rf * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred_rf))


Random Forest accuracy: 0.00%
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       2.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0



/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_cla

In [13]:
from sklearn.model_selection import GridSearchCV

# Define hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Initialize the grid search
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=2, n_jobs=-1, verbose=2)

# Fit grid search
grid_search.fit(X_train, y_train)

# Best parameters
best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")

# Evaluate best model
best_rf_model = grid_search.best_estimator_
y_pred_best_rf = best_rf_model.predict(X_test)

# Evaluate the model
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
print(f"Tuned Random Forest accuracy: {accuracy_best_rf * 100:.2f}%")


Fitting 2 folds for each of 36 candidates, totalling 72 fits
Best Parameters: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Tuned Random Forest accuracy: 0.00%
